In [31]:
import os
import pandas as pd
from sqlalchemy import create_engine, event
from sqlalchemy.engine import URL
import kaggle
import pyodbc

# Set up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'tmtuong02'
os.environ['KAGGLE_KEY'] = '3265e2b9ba6703c0f0e6fcd66d3e416f'

# Download the dataset from Kaggle
kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', path='Downloads/brazilian-ecommerce.zip', unzip=True)

# SQL Server connection setup using Windows Authentication
database_config = {
    'driver': 'ODBC Driver 17 for SQL Server',  # or any other version of the ODBC Driver for SQL Server
    'server': 'TUONG',  # e.g., 'localhost' or 'server_name\instance_name'
    'database': 'olist_ecommerce_db',
    'trusted_connection': 'yes'
}

# Create a direct connection using pyodbc to execute the CREATE DATABASE statement
connection_string = f"DRIVER={database_config['driver']};SERVER={database_config['server']};DATABASE={database_config['database']};Trusted_Connection={database_config['trusted_connection']}"
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
new_database_name = 'olist_ecommerce_db'

# # Create the new database
# cursor.execute(f"CREATE DATABASE {new_database_name}")
# connection.commit()
# cursor.close()
# connection.close()

# Update connection URL to connect to the new database
database_config['database'] = new_database_name
connection_url = URL.create(
    "mssql+pyodbc",
    host=database_config['server'],
    database=database_config['database'],
    query={
        "driver": database_config['driver'],
        "trusted_connection": database_config['trusted_connection']
    }
)

# Create a new database connection to the new database
engine = create_engine(connection_url)

# List of CSV files to import
csv_files = [
    'olist_customers_dataset.csv',
    'olist_orders_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv',
    'product_category_name_translation.csv'
]

# Import each CSV file into the new database
for file in csv_files:
    df = pd.read_csv(file)
    table_name = file.replace('.csv', '')
    df.to_sql(table_name, engine, index=False, if_exists='replace')

print("Data imported successfully!")

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Data imported successfully!
